In [ ]:
LECTURA DE PREDICCIONES DE LA APLICACION WEATHERBIT CON SU API
La api de la aplicación se puede leer con una api_key = key
Las lecturas que vamos a hacer son las predicciones a 15 días en la url "http://api.weatherbit.io/v2.0/forecast/daily"

## CREDENCIALES
en AWS tendremos las credenciales de AWS
#export AWS_ACCESS_KEY_ID=aws_access_key_id
#export AWS_SECRET_ACCESS_KEY=aws_secret_access_key
#export AWS_SESSION_TOKEN=aws_session_tokeN
##nombre del Bucket S3
#export BUCKET_NAME = BUCKET_NAME
##Api_Key de la app de Weatherbit
#export key = key


In [9]:
#Para leer las credenciales 
import os
import boto3

# Obtenemos las credenciales de las variables de entorno
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
aws_session_token = os.environ.get('AWS_SESSION_TOKEN')
#Lectura del bucket de S3
BUCKET_NAME = os.environ.get('BUCKET_NAME')
# Obtenemos las API keys de las variables de entorno
key = os.environ.get('key')


In [10]:
import requests #para trabajar con APIs
import pandas as pd #para el tratamiento de datos con pandas
import json #para trabajar con ficheros json
import boto3 #para interactuar con AWS (concretamente con S3)
import csv #para importar el csv de las estaciones

In [11]:
#Lectura de credenciales para s3
s3 = boto3.client(
 's3',
 aws_access_key_id=aws_access_key_id,
 aws_secret_access_key=aws_secret_access_key,
 aws_session_token=aws_session_token
)

Para las lecturas de la aplicación hay que indicar la latitud y la longitud de la localización.
Para ello hemos creado unos ficheros csv con las coordenadas de longituc ("lon_city.csv") y latitud ("lat_city.csv")

In [12]:
#url de Weatherbit
url = "http://api.weatherbit.io/v2.0/forecast/daily"
#lectura de longitudes
estaciones_lon = pd.read_csv('lon_city.csv') 
# lectura de latitudes
estaciones_lat = pd.read_csv('lat_city.csv')
# Incluimos los datos que leemos en una lista
lon = estaciones_lon['lon'].tolist()
lat = estaciones_lat['lat'].tolist()

In [13]:
#Para leer la fecha de hoy y añadirla al fichero y al nombre del fichero
from datetime import datetime
fecha_hoy = datetime.now()
fecha_today = fecha_hoy.strftime("%Y_%m_%d")

Hacemos un bucle para ir recorriendo todas las ciudades de la lista, que identificamos con los parámetros de longitud y latitud, que ya se encuentran en dos listas llamadas "lon" y "lat"

In [15]:
# Bucle para ir leyendo las diferentes previsiones de ciudades con lat y lon
for loni, lati in zip(lon, lat):
    params = {
        'accept': 'application/json',
        'key':key,
        'lat':lati,
        'lon':loni,
    }
    # lectura de la API con la función request
    res = requests.get(url, params=params)
    res_json = res.json()
    
    #Transformación de json a pandas
    df = pd.json_normalize(res_json['data'])
    # leemos el nombre de la ciudad correspondiente
    city_name = (res_json['city_name'])
    # Verificación de las lecturas
    print("loni:", loni, "- lati:", lati, "-city_name:", city_name) 
    
    # Incluimos una columna con el nombre de la ciudad en el dataframe
    df['city_name'] = city_name
    # Incluimos una columna con la fecha de lectura
    df['fecha_today'] = fecha_today
    # convertimos el dataframe en parquet
    df.to_parquet(f'df_{fecha_today}_{city_name}.parquet')
    
    #Almacenamos los datos en el bucket de S3 con su nombre de fecha y ciudad
    s3.upload_file(f'df_{fecha_today}_{city_name}.parquet', BUCKET_NAME, f'predicciones/df_{fecha_today}_{city_name}.parquet')


loni: 0.16667 - lati: 38.783329 -city_name: Javea
loni: -0.48149 - lati: 38.345169 -city_name: Alicante
loni: -1.04024 - lati: 38.4123 -city_name: Pinoso
loni: -0.70107 - lati: 38.26218 -city_name: Elche
loni: -0.11935 - lati: 38.919708 -city_name: Oliva
loni: -0.37334 - lati: 39.192478 -city_name: Polinyà de Xúquer
loni: -1.2 - lati: 39.566669 -city_name: Utiel
loni: -0.46349 - lati: 39.49139 -city_name: Manises
loni: -0.37739 - lati: 39.469749 -city_name: Valencia
loni: -0.51667 - lati: 38.98333 -city_name: Xàtiva
loni: -0.03333 - lati: 39.98333 -city_name: Castelló de la Plana
loni: -0.19133 - lati: 40.502079 -city_name: Castellfort
loni: -0.25774 - lati: 40.428848 -city_name: Villafranca del Cid
loni: 0.47559 - lati: 40.470329 -city_name: Vinaròs
